In [498]:
import itertools
import math
import numpy as np

In [499]:
# Source: https://www.geeksforgeeks.org/python-program-to-get-all-subsets-of-given-size-of-a-set/
def findsubsets(polygon, n):
    """
    Computes the number of subsets that can be created.

    Args:
        polygon: The shape whose subsets we are seeking.
        n: The number of elements per subset

    Returns:
        A list of subsets of with each subset having a length of "n"
    """
    return list(itertools.combinations(polygon, n))

In [500]:
def minimumSubset(A, subsets):
    """
    Finds the subset that best represents the polygon A
    
    Args:
        A: Polygon that we are seeking to replicate
        subsets: A list of subsets 

    Returns:
        A list of subsets of with each subset having a length of "n"
    """
    d_min = math.inf
    subset_min = 0
    i = 0
    while i < len(subsets):
        j = 0
        d = 0
        while j < len(A):
            result = np.array(A[j]) - np.array(subsets[i][j])
            d = d + (np.sum(np.square(result)))
            j += 1
        i += 1

        if (d < d_min):
            d_min = d
            subset_min = i
    return subset_min, d_min

In [511]:
def getEquationCoeffs(segments_A):
    """
    Finds the linear equations for the segements of a polygon. These segments are in a form of Ax+By+C=0.
    
    Args:
        segments_A: Segments of a polygon

    Returns:
        A list of the coefficients mainly, A, B and C for every linear equation formed using the segments.
    """
    equations_coeffs = [] # Ax + By + C = 0, A = index 0, B = index 1, C = index 2

    for segment in segments_A:
        if (segment[0][0] == segment[1][0]):
            equations_coeffs.append([1, 0, -segment[0][0]])
        elif (segment[0][1] == segment[1][1]):
            equations_coeffs.append([0, 1, -segment[0][1]])
        else:
            slope = (segment[1][1] - segment[0][1]) / (segment[1][0] - segment[0][0])
            y_intercept = segment[0][1] - slope * segment[0][0]
        
            # Assuming the coefficient for y will always be one, since we just rearranging y=mx+c
            # Multiplying by -1 for rearrangement purposes
            equations_coeffs.append([(slope * -1), 1, (y_intercept * -1)]) 
    return equations_coeffs

In [512]:
def findDistanceToLine(x, y, equationList):
    """
    Finds the distance from a point (x,y) to the segments of a polygon.
    
    Args:
        x: The 'x' value of the coordinate
        y: The 'y' value of the coordinate
        equationList: A list of coefficients (A, B, C) from Ax+By+C=0 for all 
                      linear equations representing the line segments of a polygon.

    Returns:
        A list of the distances from the point (x,y) to every equation.
    """
    distanceList = {}
    for coeffs in equationList:
        A, B, C = coeffs
        distance = abs((A * x + B * y + C)) / (math.sqrt(A**2 + B**2))
        distanceList[distance] = coeffs
    
    return distanceList

In [503]:
def spatialInterpolate(polygon_A, polygon_B, segments_A, segments_B):
    """
    Spatially interpolates polygon A to polygon B by adding a vertex over one frame.
    
    Args:
        polygon_A: The polygon that represents the starting polygon
        polygon_B: The polygon we want to have in our final frame
        equationList: A list of coefficients (A, B, C) from Ax+By+C=0 for all 
                      linear equations representing the line segments of a polygon.

    Returns:
        A list of the distances from the point (x,y) to every equation.
    """
        
    
    subsets = findsubsets(polygon_B, len(polygon_A))
    subset_length = len(subsets)

    s_min, distance = minimumSubset(polygon_A, subsets)
    
    b_corr = np.array(subsets[s_min]) # the best correspondence for polygon "B"
    
    # https://stackoverflow.com/questions/69435359/fast-check-if-elements-in-array-are-within-another-array-2d
    S = polygon_B[(polygon_B[:, None] == b_corr).all(-1).any(-1) == False]
    
    b_corr = np.array(b_corr)
    
    smallestValues = {}
    i = 0
    for coordinate in S:
        x, y = coordinate
        segmentDistances = findDistanceToLine(x, y, equations_coeffs)
        smallest_key = min(segmentDistances.keys())
        smallestValues[smallest_key] = segmentDistances[smallest_key]
    
    greatest_key = max(smallestValues.keys())
    greatest_segment = smallestValues[greatest_key]
    
    best_segment = segments_A[equations_coeffs.index(greatest_segment)]
        
    print("Best segment:")
    print(best_segment)
    mid_x1 = best_segment[0][0]
    mid_x2 = best_segment[1][0]

    mid_y1 = best_segment[0][1]
    mid_y2 = best_segment[1][1]
    
    midpoint_x = (mid_x1 + mid_x2)/2 
    midpoint_y = (mid_y1 + mid_y2)/2
    
    newPoint = [midpoint_x, midpoint_y]
    
    
    # Add the new segments connecting the midpoint
    segments_A = segments_A.tolist() + [[best_segment[0].tolist(), newPoint]] + [[best_segment[1].tolist(), newPoint]]

    segments_A.remove(segments_A[equations_coeffs.index(greatest_segment)])
    # Add new midpoint to polygon
    polygon_A = np.array(polygon_A.tolist() + [newPoint])
    
    # Remove the best segment since its been split in two

    
    return polygon_A, np.array(segments_A)

In [504]:
segments_A

array([[[0. , 0. ],
        [0. , 2. ]],

       [[0. , 2. ],
        [1. , 0. ]],

       [[1. , 0. ],
        [0. , 0. ]],

       [[1. , 0. ],
        [0.5, 0. ]],

       [[0. , 0. ],
        [0.5, 0. ]]])

In [505]:
polygon_A = np.array([[0,0],[1,0],[0,2]]) # Polygon A
polygon_B = np.array([[5,0], [0,4], [6,5], [7,5], [7,0]]) # Polygon B
segments_A = np.array([[[0,0], [0,2]], [[0,2], [1,0]], [[1,0], [0,0]]])
segments_B = np.array([[[5,0], [0,4]], [[0,4], [6,5]], [[6,5], [7,5]], [[7,5], [7,0]], [[7,0], [5,0]]])
x = 3 # Number of frames

for i in range(1, x):
    polygon_A, segments_A = spatialInterpolate(polygon_A, polygon_B, segments_A, segments_B)
    print("Frame ", i, ": ", "Polygon -> ", polygon_A.tolist(), " Segments -> ", segments_A.tolist())

Best segment:
[[1 0]
 [0 0]]
Frame  1 :  Polygon ->  [[0.0, 0.0], [1.0, 0.0], [0.0, 2.0], [0.5, 0.0]]  Segments ->  [[[0.0, 0.0], [0.0, 2.0]], [[0.0, 2.0], [1.0, 0.0]], [[1.0, 0.0], [0.5, 0.0]], [[0.0, 0.0], [0.5, 0.0]]]
Best segment:
[[1.  0. ]
 [0.5 0. ]]
Frame  2 :  Polygon ->  [[0.0, 0.0], [1.0, 0.0], [0.0, 2.0], [0.5, 0.0], [0.75, 0.0]]  Segments ->  [[[0.0, 0.0], [0.0, 2.0]], [[0.0, 2.0], [1.0, 0.0]], [[0.0, 0.0], [0.5, 0.0]], [[1.0, 0.0], [0.75, 0.0]], [[0.5, 0.0], [0.75, 0.0]]]


In [506]:
subsets = [([5,0], [0,4], [6,5], [7,5]), ([5,0], [0,4], [6,5], [7,0]), ([5,0], [0,4], [7,5], [7,0]), ([5,0], [6,5], [7,5], [7,0]), ([0,4], [6,5], [7,5], [7,0])]

In [507]:
polygon_A = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 2.0], [0.5, 0.0]])

In [508]:
segments_A = np.array([[[0.0, 0.0], [0.0, 2.0]], [[0.0, 2.0], [1.0, 0.0]], [[1.0, 0.0], [0.0, 0.0]], [[1.0, 0.0], [0.5, 0.0]], [[0.0, 0.0], [0.5, 0.0]]])

In [509]:
s_min, distance = minimumSubset(polygon_A, subsets)

In [510]:
spatialInterpolate(polygon_A, polygon_B, segments_A, segments_B)

Best segment:
[[1. 0.]
 [0. 0.]]


(array([[0. , 0. ],
        [1. , 0. ],
        [0. , 2. ],
        [0.5, 0. ],
        [0.5, 0. ]]),
 array([[[0. , 0. ],
         [0. , 2. ]],
 
        [[0. , 2. ],
         [1. , 0. ]],
 
        [[1. , 0. ],
         [0.5, 0. ]],
 
        [[0. , 0. ],
         [0.5, 0. ]],
 
        [[1. , 0. ],
         [0.5, 0. ]],
 
        [[0. , 0. ],
         [0.5, 0. ]]]))